In [ ]:
""" Trains an agent with (stochastic) Policy Gradients on Pong. Uses OpenAI Gym. """
import numpy as np
import pickle
import gym

# hyperparameters
H = 200 # number of hidden layer neurons
batch_size = 10 # every how many episodes to do a param update?
learning_rate = 1e-4
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
resume = False # resume from previous checkpoint?
render = False

# model initialization
D = 80 * 80 # input dimensionality: 80x80 grid
if resume:
  model = pickle.load(open('save.p', 'rb'))
else:
  model = {}
  model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
  model['W2'] = np.random.randn(H) / np.sqrt(H)
  
grad_buffer = { k : np.zeros_like(v) for k,v in model.items() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.items() } # rmsprop memory

def sigmoid(x): 
  return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
  """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
  I = I[35:195] # crop
  I = I[::2,::2,0] # downsample by factor of 2
  I[I == 144] = 0 # erase background (background type 1)
  I[I == 109] = 0 # erase background (background type 2)
  I[I != 0] = 1 # everything else (paddles, ball) just set to 1
  return I.astype(np.float).ravel()

def discount_rewards(r):
  """ take 1D float array of rewards and compute discounted reward """
  discounted_r = np.zeros_like(r)
  running_add = 0
  for t in reversed(range(0, r.size)):
    if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
    running_add = running_add * gamma + r[t]
    discounted_r[t] = running_add
  return discounted_r

def policy_forward(x):
  h = np.dot(model['W1'], x)
  h[h<0] = 0 # ReLU nonlinearity
  logp = np.dot(model['W2'], h)
  p = sigmoid(logp)
  return p, h # return probability of taking action 2, and hidden state

def policy_backward(eph, epdlogp):
  """ backward pass. (eph is array of intermediate hidden states) """
  dW2 = np.dot(eph.T, epdlogp).ravel()
  dh = np.outer(epdlogp, model['W2'])
  dh[eph <= 0] = 0 # backpro prelu
  dW1 = np.dot(dh.T, epx)
  return {'W1':dW1, 'W2':dW2}

env = gym.make("Pong-v0")
observation = env.reset()
prev_x = None # used in computing the difference frame
xs,hs,dlogps,drs = [],[],[],[]
running_reward = None
reward_sum = 0
episode_number = 0
while True:
  if render: env.render()

  # preprocess the observation, set input to network to be difference image
  cur_x = prepro(observation)
  x = cur_x - prev_x if prev_x is not None else np.zeros(D)
  prev_x = cur_x

  # forward the policy network and sample an action from the returned probability
  aprob, h = policy_forward(x)
  action = 2 if np.random.uniform() < aprob else 3 # roll the dice!

  # record various intermediates (needed later for backprop)
  xs.append(x) # observation
  hs.append(h) # hidden state
  y = 1 if action == 2 else 0 # a "fake label"
  dlogps.append(y - aprob) # grad that encourages the action that was taken to be taken (see http://cs231n.github.io/neural-networks-2/#losses if confused)

  # step the environment and get new measurements
  observation, reward, done, info = env.step(action)
  reward_sum += reward

  drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

  if done: # an episode finished
    episode_number += 1

    # stack together all inputs, hidden states, action gradients, and rewards for this episode
    epx = np.vstack(xs)
    eph = np.vstack(hs)
    epdlogp = np.vstack(dlogps)
    epr = np.vstack(drs)
    xs,hs,dlogps,drs = [],[],[],[] # reset array memory

    # compute the discounted reward backwards through time
    discounted_epr = discount_rewards(epr)
    # standardize the rewards to be unit normal (helps control the gradient estimator variance)
    discounted_epr -= np.mean(discounted_epr)
    discounted_epr /= np.std(discounted_epr)

    epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
    grad = policy_backward(eph, epdlogp)
    for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch

    # perform rmsprop parameter update every batch_size episodes
    if episode_number % batch_size == 0:
      for k,v in model.items():
        g = grad_buffer[k] # gradient
        rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
        model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
        grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

    # boring book-keeping
    running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
    print('resetting env. episode reward total was %f. running mean: %f' % (reward_sum, running_reward))
    if episode_number % 100 == 0: pickle.dump(model, open('save.p', 'wb'))
    reward_sum = 0
    observation = env.reset() # reset env
    prev_x = None

    if reward != 0: # Pong has either +1 or -1 reward exactly when game ends.
        print('ep %d: game finished, reward: %f' % (episode_number, reward) + ('' if reward == -1 else ' !!!!!!!!'))

resetting env. episode reward total was -20.000000. running mean: -20.000000
ep 1: game finished, reward: -1.000000
resetting env. episode reward total was -20.000000. running mean: -20.000000
ep 2: game finished, reward: -1.000000
resetting env. episode reward total was -17.000000. running mean: -19.970000
ep 3: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -19.980300
ep 4: game finished, reward: -1.000000
resetting env. episode reward total was -20.000000. running mean: -19.980497
ep 5: game finished, reward: -1.000000
resetting env. episode reward total was -20.000000. running mean: -19.980692
ep 6: game finished, reward: -1.000000
resetting env. episode reward total was -20.000000. running mean: -19.980885
ep 7: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -19.991076
ep 8: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.00

resetting env. episode reward total was -20.000000. running mean: -20.160324
ep 72: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.168720
ep 73: game finished, reward: -1.000000
resetting env. episode reward total was -18.000000. running mean: -20.147033
ep 74: game finished, reward: -1.000000
resetting env. episode reward total was -18.000000. running mean: -20.125563
ep 75: game finished, reward: -1.000000
resetting env. episode reward total was -19.000000. running mean: -20.114307
ep 76: game finished, reward: -1.000000
resetting env. episode reward total was -20.000000. running mean: -20.113164
ep 77: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.122032
ep 78: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.130812
ep 79: game finished, reward: -1.000000
resetting env. episode reward total was -20.000000. running mean

resetting env. episode reward total was -18.000000. running mean: -20.281899
ep 142: game finished, reward: -1.000000
resetting env. episode reward total was -18.000000. running mean: -20.259080
ep 143: game finished, reward: -1.000000
resetting env. episode reward total was -17.000000. running mean: -20.226489
ep 144: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.234224
ep 145: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.241882
ep 146: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.249463
ep 147: game finished, reward: -1.000000
resetting env. episode reward total was -19.000000. running mean: -20.236969
ep 148: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.244599
ep 149: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. runn

resetting env. episode reward total was -20.000000. running mean: -20.346338
ep 212: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.352875
ep 213: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.359346
ep 214: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.365753
ep 215: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.372095
ep 216: game finished, reward: -1.000000
resetting env. episode reward total was -20.000000. running mean: -20.368374
ep 217: game finished, reward: -1.000000
resetting env. episode reward total was -20.000000. running mean: -20.364690
ep 218: game finished, reward: -1.000000
resetting env. episode reward total was -20.000000. running mean: -20.361044
ep 219: game finished, reward: -1.000000
resetting env. episode reward total was -20.000000. runn

resetting env. episode reward total was -21.000000. running mean: -20.445621
ep 282: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.451165
ep 283: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.456653
ep 284: game finished, reward: -1.000000
resetting env. episode reward total was -20.000000. running mean: -20.452087
ep 285: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.457566
ep 286: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.462990
ep 287: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.468360
ep 288: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.473677
ep 289: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. runn

...

resetting env. episode reward total was -21.000000. running mean: -20.404946
ep 772: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.410896
ep 773: game finished, reward: -1.000000
resetting env. episode reward total was -20.000000. running mean: -20.406787
ep 774: game finished, reward: -1.000000
resetting env. episode reward total was -19.000000. running mean: -20.392719
ep 775: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.398792
ep 776: game finished, reward: -1.000000
resetting env. episode reward total was -20.000000. running mean: -20.394804
ep 777: game finished, reward: -1.000000
resetting env. episode reward total was -19.000000. running mean: -20.380856
ep 778: game finished, reward: -1.000000
resetting env. episode reward total was -20.000000. running mean: -20.377048
ep 779: game finished, reward: -1.000000
resetting env. episode reward total was -20.000000. runn

resetting env. episode reward total was -21.000000. running mean: -20.299244
ep 842: game finished, reward: -1.000000
resetting env. episode reward total was -20.000000. running mean: -20.296251
ep 843: game finished, reward: -1.000000
resetting env. episode reward total was -20.000000. running mean: -20.293289
ep 844: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.300356
ep 845: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.307352
ep 846: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.314279
ep 847: game finished, reward: -1.000000
resetting env. episode reward total was -20.000000. running mean: -20.311136
ep 848: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.318025
ep 849: game finished, reward: -1.000000
resetting env. episode reward total was -19.000000. runn

resetting env. episode reward total was -21.000000. running mean: -20.315197
ep 912: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.322045
ep 913: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.328824
ep 914: game finished, reward: -1.000000
resetting env. episode reward total was -19.000000. running mean: -20.315536
ep 915: game finished, reward: -1.000000
resetting env. episode reward total was -19.000000. running mean: -20.302381
ep 916: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.309357
ep 917: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.316263
ep 918: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.323101
ep 919: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. runn

resetting env. episode reward total was -19.000000. running mean: -20.349805
ep 982: game finished, reward: -1.000000
resetting env. episode reward total was -20.000000. running mean: -20.346307
ep 983: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.352844
ep 984: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.359316
ep 985: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.365722
ep 986: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.372065
ep 987: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.378345
ep 988: game finished, reward: -1.000000
resetting env. episode reward total was -20.000000. running mean: -20.374561
ep 989: game finished, reward: -1.000000
resetting env. episode reward total was -19.000000. runn

resetting env. episode reward total was -20.000000. running mean: -20.230827
ep 1051: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.238518
ep 1052: game finished, reward: -1.000000
resetting env. episode reward total was -19.000000. running mean: -20.226133
ep 1053: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.233872
ep 1054: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.241533
ep 1055: game finished, reward: -1.000000
resetting env. episode reward total was -20.000000. running mean: -20.239118
ep 1056: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -20.246727
ep 1057: game finished, reward: -1.000000
resetting env. episode reward total was -20.000000. running mean: -20.244259
ep 1058: game finished, reward: -1.000000
resetting env. episode reward total was -18.0000